# Split & Sampling Notebook — Rubens vs Picasso

Deze notebook maakt **train/val/test** datasets aan voor het 2-klassen experiment **Rubens vs Picasso**, met drie varianten:
- **imbalanced** (originele verdeling)
- **undersampled** (beide klassen even groot = kleinste klasse)
- **oversampled** (train-set gebalanceerd door duplicatie)

📌 **Input (brondata):** `schilderijen/` (hier zitten je opgeschoonde JPG+RGB afbeeldingen)

📌 **Output (trainbare datasets):** `datasets/rubens_picasso/<variant>/{train,val,test}/<klasse>/...`

⚠️ Belangrijk: de **testset** blijft altijd gebaseerd op unieke originele afbeeldingen (geen duplicaten), zodat evaluatie eerlijk blijft.


## 0. Imports & configuratie

In [ ]:
from pathlib import Path
import random
import shutil
from collections import defaultdict

# ====== PAS AAN ALS JE PADEN ANDERS HETEN ======
SRC_ROOT = Path("schilderijen")            # bron: opgeschoonde data
OUT_ROOT = Path("datasets/rubens_picasso") # output: train/val/test folders

CLASSES = ["Rubens", "Picasso"]

# Hold-out split (train/val/test)
SPLIT = {"train": 0.70, "val": 0.15, "test": 0.15}

# Reproduceerbaarheid
SEED = 42
random.seed(SEED)

# Veiligheid: als True -> bestaande outputmap verwijderen en opnieuw maken
OVERWRITE = False


## 1. Snelle checks
We controleren of de bronmappen bestaan en hoeveel afbeeldingen er per klasse aanwezig zijn.

In [ ]:
def list_images(class_name: str):
    """Geef alle JPG bestanden terug voor een klasse."""
    class_dir = SRC_ROOT / class_name
    return sorted([p for p in class_dir.glob("*.jpg") if p.is_file()])

# Check bronmappen
for c in CLASSES:
    assert (SRC_ROOT / c).exists(), f"Bronmap ontbreekt: {SRC_ROOT / c}"

files_by_class = {c: list_images(c) for c in CLASSES}
counts = {c: len(files_by_class[c]) for c in CLASSES}

print("Aantal afbeeldingen per klasse (brondata):")
for c in CLASSES:
    print(f" - {c}: {counts[c]}")

min_n = min(counts.values())
max_n = max(counts.values())

print(f"\nmin_n (kleinste klasse) = {min_n}")
print(f"max_n (grootste klasse)  = {max_n}")


## 2. Hulpfuncties
We maken functies om:
- mappen te (her)maken
- een hold-out split uit te voeren
- bestanden te kopiëren (met unieke naam indien duplicaten nodig zijn)


In [ ]:
def reset_output_folder(path: Path, overwrite: bool = False):
    """Maak output folder leeg (optioneel) en zet basisstructuur klaar."""
    if path.exists() and overwrite:
        shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

def ensure_split_dirs(base: Path, classes=CLASSES):
    """Maak train/val/test + class subfolders aan."""
    for split in ["train", "val", "test"]:
        for c in classes:
            (base / split / c).mkdir(parents=True, exist_ok=True)

def holdout_split(file_list, split=SPLIT):
    """Shuffle en split één lijst in train/val/test."""
    file_list = file_list.copy()
    random.shuffle(file_list)
    n = len(file_list)
    n_train = int(n * split["train"])
    n_val   = int(n * split["val"])
    train = file_list[:n_train]
    val   = file_list[n_train:n_train+n_val]
    test  = file_list[n_train+n_val:]
    return {"train": train, "val": val, "test": test}

def copy_files(files, dst_dir: Path, allow_duplicates: bool = False):
    """Kopieer files naar dst_dir.

    - allow_duplicates=False: verwacht unieke filenames; overwrite is niet de bedoeling.
    - allow_duplicates=True: als dezelfde file (of naam) meerdere keren wordt gebruikt, 
      maken we een unieke bestandsnaam zodat niets overschreven wordt.
    """
    dst_dir.mkdir(parents=True, exist_ok=True)
    dup_counter = defaultdict(int)

    for f in files:
        name = f.name
        target = dst_dir / name

        if allow_duplicates and target.exists():
            # maak unieke naam: <stem>__dup0001.jpg
            dup_counter[name] += 1
            target = dst_dir / f"{f.stem}__dup{dup_counter[name]:04d}{f.suffix}"

        shutil.copy2(f, target)


## 3. Variant A — Imbalanced dataset
We behouden de **originele verdeling** en maken een hold-out split per klasse.

In [ ]:
# Outputmap voor imbalanced
imbalanced_dir = OUT_ROOT / "imbalanced"

reset_output_folder(imbalanced_dir, overwrite=OVERWRITE)
ensure_split_dirs(imbalanced_dir)

splits_imbalanced = {c: holdout_split(files_by_class[c]) for c in CLASSES}

# Kopieer bestanden
for c in CLASSES:
    for split_name, flist in splits_imbalanced[c].items():
        copy_files(flist, imbalanced_dir / split_name / c, allow_duplicates=False)

print("✅ Imbalanced dataset aangemaakt:", imbalanced_dir)


## 4. Variant B — Undersampled dataset
We maken de dataset **gebalanceerd** door per klasse exact `min_n` afbeeldingen te nemen (de grootte van de kleinste klasse), en doen daarna een hold-out split.

📌 Voor Rubens vs Picasso betekent dit: beide klassen worden teruggebracht tot `min_n`.

In [ ]:
undersampled_dir = OUT_ROOT / "undersampled"

reset_output_folder(undersampled_dir, overwrite=OVERWRITE)
ensure_split_dirs(undersampled_dir)

# Kies per klasse exact min_n unieke afbeeldingen
undersampled_pool = {}
for c in CLASSES:
    undersampled_pool[c] = random.sample(files_by_class[c], min_n)

# Split de gebalanceerde set
splits_undersampled = {c: holdout_split(undersampled_pool[c]) for c in CLASSES}

# Kopieer bestanden
for c in CLASSES:
    for split_name, flist in splits_undersampled[c].items():
        copy_files(flist, undersampled_dir / split_name / c, allow_duplicates=False)

print("✅ Undersampled dataset aangemaakt:", undersampled_dir)


## 5. Variant C — Oversampled dataset (train-set balancing)
Bij oversampling vullen we de **kleinere klasse aan** met duplicaten zodat de **train-set** evenveel voorbeelden heeft als de grootste klasse.

Waarom enkel de **train-set** oversamplen?
- De **validatie** en **test** willen we zo 'echt' mogelijk houden (unieke originele beelden).
- Duplicaten in test/val zouden de evaluatie vertekenen.

Dus:
- `val` en `test` = gewone hold-out uit originele data
- `train` = oversampled tot dezelfde grootte per klasse


In [ ]:
oversampled_dir = OUT_ROOT / "oversampled"

reset_output_folder(oversampled_dir, overwrite=OVERWRITE)
ensure_split_dirs(oversampled_dir)

# Start van een gewone hold-out split (originele data)
splits_base = {c: holdout_split(files_by_class[c]) for c in CLASSES}

# Bepaal gewenste train-size (= max van train sizes)
train_sizes = {c: len(splits_base[c]["train"]) for c in CLASSES}
target_train_n = max(train_sizes.values())

print("Train sizes (base):", train_sizes)
print("Target train size per klasse (oversampled):", target_train_n)

splits_oversampled = {c: {"train": [], "val": [], "test": []} for c in CLASSES}

for c in CLASSES:
    # val/test blijven uniek
    splits_oversampled[c]["val"] = splits_base[c]["val"]
    splits_oversampled[c]["test"] = splits_base[c]["test"]

    base_train = splits_base[c]["train"]
    if len(base_train) == target_train_n:
        splits_oversampled[c]["train"] = base_train
    else:
        # Oversample with replacement uit de base_train set
        # (in praktijk combineer je dit later met data augmentation om duplicaten te 'maskeren')
        needed = target_train_n - len(base_train)
        extra = [random.choice(base_train) for _ in range(needed)]
        splits_oversampled[c]["train"] = base_train + extra

# Kopieer bestanden; allow_duplicates=True voor train (anders overschrijven duplicaten)
for c in CLASSES:
    copy_files(splits_oversampled[c]["train"], oversampled_dir / "train" / c, allow_duplicates=True)
    copy_files(splits_oversampled[c]["val"],   oversampled_dir / "val"   / c, allow_duplicates=False)
    copy_files(splits_oversampled[c]["test"],  oversampled_dir / "test"  / c, allow_duplicates=False)

print("✅ Oversampled dataset aangemaakt:", oversampled_dir)


## 6. Controle: aantallen per split en per klasse
We controleren of de outputmappen de verwachte aantallen bevatten.

In [ ]:
def count_images(base: Path):
    counts = {}
    for split in ["train", "val", "test"]:
        counts[split] = {}
        for c in CLASSES:
            counts[split][c] = len(list((base / split / c).glob("*.jpg")))
    return counts

def print_counts(title: str, base: Path):
    c = count_images(base)
    print(f"\n=== {title} ===")
    for split in ["train", "val", "test"]:
        print(f"{split}: ", {k: v for k, v in c[split].items()})

print_counts("IMBALANCED", imbalanced_dir)
print_counts("UNDERSAMPLED", undersampled_dir)
print_counts("OVERSAMPLED", oversampled_dir)


## 7. (Optioneel) Visualiseer enkele afbeeldingen per split
Kleine sanity check: tonen van een paar willekeurige afbeeldingen per klasse.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random

def show_random_images(base: Path, split: str, class_name: str, n=3):
    files = list((base / split / class_name).glob("*.jpg"))
    if len(files) == 0:
        print("Geen files gevonden.")
        return
    sample = random.sample(files, min(n, len(files)))

    plt.figure(figsize=(12, 4))
    for i, f in enumerate(sample, 1):
        img = Image.open(f)
        plt.subplot(1, len(sample), i)
        plt.imshow(img)
        plt.title(f.name)
        plt.axis("off")
    plt.show()

# Voorbeeld: 3 willekeurige Rubens in train van undersampled variant
show_random_images(undersampled_dir, split="train", class_name="Rubens", n=3)


## 8. Wat hierna?
Nu je datasets klaar staan, kan je meteen trainen met Keras:

```python
train_ds = tf.keras.utils.image_dataset_from_directory(
    'datasets/rubens_picasso/undersampled/train',
    image_size=(224,224),
    batch_size=32
)
```

Aanbevolen volgorde (zoals in het boek/les):
1) Eenvoudige CNN baseline
2) CNN + data augmentation
3) VGG16 feature extraction
4) VGG16 + augmentation
5) Fine-tuning
